In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics, rich
from dataclasses import dataclass, field
from IPython.display import display

In [190]:
@dataclass
class Handshake:
    _df: pd.DataFrame
    start: float
    end: float

    @property
    def df(self):
        return self._df.iloc[self.start:self.end]

    def with_offset(self, offset):
        return Handshake(self._df, self.start-offset, self.end+offset)

    def with_lines_offset_se(self, offset_s, offset_e):
        return Handshake(self._df, self.start-offset_s, self.end+offset_e)

    @property
    def start_ms(self):
        return self._df['timestamp'].iloc[self.start]

    @property
    def end_ms(self):
        return self._df['timestamp'].iloc[self.end]

    @property
    def duration_ms(self): # in milliseconds
        return round((self.end_ms - self.start_ms) * 1000, 2)

    @property
    def energy(self): # in millijoules
        time_step = 0.000250 # seconds
        joules = (self.df['power'] * time_step).sum()
        return round(joules * 1000, 2)

    @property
    def power_avg(self):
        return self.df['power'].mean()

    def __str__(self) -> str:
        return f"""Duration (ms): {self.duration_ms}
Energy (mJ): {self.energy}"""

    def __repr__(self) -> str:
        dic = {
            "duration": self.duration_ms,
            "energy": self.energy,
        }
        return str(dic)

class DataLoader:
    csv_files_otii = {
        "current": "Main current - Arc.csv",
        "power": "Main power - Arc.csv",
        "gpio": "GPI 1 - Arc.csv",
    }

    # @lru_cache
    def find_handshakes(df):
        # convert NaN to an arbitrary integer
        df['gpio'] = df['gpio'].fillna(-9).astype(int)

        gpio_values = df['gpio'].values
        in_handshake = False
        hs_start, hs_end = 0, 0
        handshakes = []
        for index, value in enumerate(gpio_values):
            if not in_handshake and value == 1:
                in_handshake = True
                hs_start = index
            elif in_handshake and value == 0:
                in_handshake = False
                hs_end = index
                hs = Handshake(df, hs_start, hs_end)
                handshakes.append(hs)

            if in_handshake:
                gpio_values[index] = 1

        # convert temporary value to 0
        gpio_values[gpio_values == -9] = 0
        # Update the 'gpio' column in the DataFrame
        df['gpio'] = gpio_values
        return df, handshakes

    def run(results_dir):
        dfs = []

        for source, csv_file in DataLoader.csv_files_otii.items():
            filename = f"{results_dir}/{csv_file}"
            df = pd.read_csv(filename)
            df = df.rename(columns={'Timestamp': 'timestamp'})
            df = df.rename(columns={'Value': source})
            # print(df.head())
            dfs.append(df)

        merged_df = dfs[0]
        for df in dfs[1:]:
            merged_df = merged_df.merge(df, on="timestamp", how="outer")

        merged_df, handshakes = DataLoader.find_handshakes(merged_df)

        return merged_df, handshakes

def mean_stdev(arr, ndigits=2):
    return round(statistics.mean(arr), ndigits), round(statistics.stdev(arr), ndigits)

@dataclass
class HandshakeSet:
    label: str
    csv_folder: str
    df: pd.DataFrame = field(default_factory=pd.DataFrame)
    handshakes: list = field(default_factory=list)

    def __post_init__(self):
        self.df, self.handshakes = DataLoader.run(self.csv_folder)

    def __getitem__(self, index):
        return self.handshakes[index]

    def duration_mean_stdev(self):
        return mean_stdev([hs.duration_ms for hs in self.handshakes])

    def energy_mean_stdev(self):
        return mean_stdev([hs.energy for hs in self.handshakes])

    def power_mean(self):
        return round(statistics.mean([hs.power_avg for hs in self.handshakes]), 2)

    def __repr__(self) -> str:
        return str(self.summary())

    def summary(self):
        return {
            'label': self.label,
            'handshakes': len(self.handshakes),
            'duration': self.duration_mean_stdev(),
            'energy': self.energy_mean_stdev(),
            'power': self.power_mean(),
        }

@dataclass
class HandshakeMemory():
    csv_file: str
    df: pd.DataFrame = None

    def __post_init__(self):
        self.df = pd.read_csv(self.csv_file).set_index('protocol')
        self.df = (self.df / 1000) #.astype(int) # convert to kB
        self.df['Flash'] = self.df[['text', 'data']].sum(axis=1)
        self.df['RAM'] = self.df[['data', 'bss', 'stack']].sum(axis=1)

    def plot(self):
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
        colors = {
            'text': 'tab:blue',
            'data': 'firebrick',
            'bss': 'tab:green', 
            'stack': 'tab:orange'
        }
        xtick_labels = ['EDHOC #20', 'DTLS 1.3']

        self.df[['data', 'bss', 'stack']].plot(kind='bar', stacked=True, ax=ax1, zorder=5, color=[colors['data'], colors['bss'], colors['stack']])
        ax1.set_xticklabels(xtick_labels, rotation=0)  # rotation=0 makes labels horizontal
        ax1.set_title('RAM Usage')
        ax1.set_xlabel('Protocol')
        ax1.set_ylabel('RAM Usage (kB)')

        self.df[['text', 'data']].plot(kind='bar', stacked=True, ax=ax2, zorder=5, color=[colors['text'], colors['data']])
        ax2.set_xticklabels(xtick_labels, rotation=0)
        ax2.set_title('Flash Usage')
        ax2.set_xlabel('Protocol')
        ax2.set_ylabel('Flash Usage (kB)')

        [ax.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5, zorder=1) for ax in [ax1, ax2]]

        plt.subplots_adjust(wspace=0.3)
        # plt.tight_layout()
        plt.show()

    def percentage_of(self, p1='edhoc', p2='dtls'):
        return {
            'RAM': round(self.df.loc[p1, 'RAM'] / self.df.loc[p2, 'RAM'] * 100, 2),
            'Flash': round(self.df.loc[p1, 'Flash'] / self.df.loc[p2, 'Flash'] * 100, 2),
        }

    def summary(self):
        return {
            'text': self.df['text'].to_dict(),
            'data': self.df['data'].to_dict(),
            'bss': self.df['bss'].to_dict(),
            'stack': self.df['stack'].to_dict(),
        }

@dataclass
class Experiment:
    label: str
    csv_folder_edhoc: str
    csv_folder_dtls: str
    memory_csv_file: str
    edhoc_hs: HandshakeSet = None
    dtls_hs: HandshakeSet = None
    memory: HandshakeMemory = None

    def __post_init__(self):
        self.edhoc_hs = HandshakeSet("edhoc", self.csv_folder_edhoc)
        self.dtls_hs = HandshakeSet("dtls", self.csv_folder_dtls)
        self.memory = HandshakeMemory(self.memory_csv_file)

    def plot_durations(self):
        protocols_names = ['EDHOC #20', 'DTLS 1.3']
        df = self.summary_handshakes_df()

        # Plotting
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
        bar_positions = [0, 0.5]

        df['duration_avg'].plot(kind='bar', ax=ax1, yerr=df['duration_std'], capsize=10, zorder=5)
        ax1.set_xticklabels(protocols_names, rotation=0)
        ax1.set_ylabel('Handshake Duration (ms)')
        ax1.set_title('Handshake Duration')

        df['energy_avg'].plot(kind='bar', ax=ax2, yerr=df['energy_std'], capsize=10, zorder=5)
        ax2.set_xticklabels(protocols_names, rotation=0)
        ax2.set_ylabel('Handshake Energy (mJ)')
        ax2.set_title('Handshake Energy Consumption')

        [ax.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5, zorder=1) for ax in [ax1, ax2]]

        plt.subplots_adjust(wspace=0.2)
        # plt.tight_layout()
        plt.show()

    def plot_handshake_instance(hs, protocol, index, os=200, oe=200):
        offset_hs = hs.with_lines_offset_se(os, oe)

        # compute offset difference to milliseconds
        offset_start_ms = hs.start_ms - offset_hs.start_ms
        offset_end_ms = offset_start_ms + (hs.duration_ms / 1000)

        offset_hs._df = offset_hs._df.copy()
        offset_hs._df['timestamp'] = offset_hs._df['timestamp'] - offset_hs.start_ms

        # orig_start_ms

        plt.figure(figsize=(18, 2))
        plt.plot(offset_hs.df['timestamp'], offset_hs.df['power'], label='Power (mW)')

        # draw vertical red lines at start and stop
        plt.axvline(x=offset_start_ms, color='red')
        plt.axvline(x=offset_end_ms, color='red')

        # print(offset_hs.df['timestamp'].min(), offset_hs.df['timestamp'].max())
        plt.xlim(0, 0.33) # this should be automated from min and max, but it is not working

        # add text above
        duration_position_x = (offset_hs.start_ms + offset_hs.end_ms) / 2
        y_position = plt.ylim()[1] * 1.03
        plt.text(duration_position_x, y_position, f"{protocol} -- handshake #{index}\n{hs}", ha='center')

        plt.xlabel('Timestamp (s)')
        plt.ylabel('Power (mW)')
        plt.grid(True, which='both', axis='y', linestyle='--', linewidth=0.5, zorder=1)

        plt.legend()
        plt.show()

    def plot_handshake_instance_comparison(edhoc_hs, dtls_hs, index):
        offset_for_larger_left = offset_for_smaller_left = 100
        offset_for_larger_right = 200
        offset_for_smaller_right = ((dtls_hs.end - dtls_hs.start) - (edhoc_hs.end - edhoc_hs.start)) + offset_for_larger

        Experiment.plot_handshake_instance(edhoc_hs, "EDHOC #20", index, offset_for_smaller_left, offset_for_smaller_right)
        Experiment.plot_handshake_instance(dtls_hs, "DTLS 1.3", index, offset_for_larger_left, offset_for_larger_right)

    def summary_handshakes_df(self):
        edhoc_sum = self.edhoc_hs.summary()
        dtls_sum = self.dtls_hs.summary()
        df = pd.DataFrame({
            'protocol': ['edhoc', 'dtls'],
            'handshakes': [edhoc_sum['handshakes'], dtls_sum['handshakes']],
            'duration_avg': [edhoc_sum['duration'][0], dtls_sum['duration'][0]],
            'duration_std': [edhoc_sum['duration'][1], dtls_sum['duration'][1]],
            'energy_avg': [edhoc_sum['energy'][0], dtls_sum['energy'][0]],
            'energy_std': [edhoc_sum['energy'][1], dtls_sum['energy'][1]],
            'power_avg': [edhoc_sum['power'], dtls_sum['power']],
        })
        df.set_index('protocol', inplace=True)
        return df

    def handshakes_percentage_of(self, p1='edhoc', p2='dtls'):
        df = self.summary_handshakes_df()
        return {
            'Duration': round(df.loc[p1, 'duration_avg'] / df.loc[p2, 'duration_avg'] * 100, 2),
            'Energy': round(df.loc[p1, 'energy_avg'] / df.loc[p2, 'energy_avg'] * 100, 2),
        }

    def summary(self):
        return {
            'label': self.label,
            'edhoc_hs': self.edhoc_hs.summary(),
            'dtls_hs': self.dtls_hs.summary(),
            'memory': self.memory.summary(),
        }

    def results(self):
        print("Handshake duration and energy (ms, mJ, mW): ")
        display(self.summary_handshakes_df())
        rich.print("EDHOC percentage of DTLS: ", self.handshakes_percentage_of())
        print("Chart: ")
        self.plot_durations()
        print("Memory usage (kB): ")
        display(self.memory.df)
        rich.print("EDHOC percentage of DTLS: ", self.memory.percentage_of())
        print("Chart: ")
        self.memory.plot()
        print("A peek at one particular handshake (red lines mean start/stop of the handshake procedure): ")
        Experiment.plot_handshake_instance_comparison(self.edhoc_hs[0], self.dtls_hs[0], 0)


In [191]:
experiment = Experiment(
    "edhoc_vs_dtls-09aug",
    "./results/edhoc-21aug-16h22-csv",
    "./results/dtls-21aug-16h26-csv",
    "./results/memory-2023-08-21_15:55:20.csv",
)

experiment.results()

Handshake duration and energy (ms, mJ, mW): 


,handshakes,duration_avg,duration_std,energy_avg,energy_std,power_avg
protocol,,,,,,
edhoc,20,183.1,2.47,6.34,0.06,0.03
dtls,20,257.4,2.30,9.10,0.06,0.04


EDHOC percentage of DTLS: 
{'Duration': 71.13, 'Energy': 69.67}

Chart: 
Memory usage (kB): 


,text,data,bss,stack,Flash,RAM
protocol,,,,,,
edhoc,141.032,0.836,45.844,9.756,141.868,56.436
dtls,260.800,0.676,184.544,6.868,261.476,192.088


EDHOC percentage of DTLS: 
{'RAM': 29.38, 'Flash': 54.26}

Chart: 
A peek at one particular handshake (red lines mean start/stop of the handshake procedure): 
